Install Python to read the environment variables from a .env file

In [1]:
! pip install python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Install Neo4j driver

In [2]:
! pip install neo4j


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Load `.env` variables

In [3]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

load_dotenv()
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")

Create driver and define `execute_query()` method

In [4]:
driver = GraphDatabase.driver(uri, auth=(user, password))
def execute_query(query):
    with driver.session() as session:
        results = session.run(query)
        return [record for record in results]

##### Q1: Given an artist, return tracks

In [5]:
q1_query = """
MATCH (track:Track)-[:COMPOSED_BY]->(artist:Artist {name: 'Nathaniel Bassey'})
RETURN track.name AS TrackName
"""
q1_results = execute_query(q1_query)
print("Q1 Results:", q1_results)

Q1 Results: [<Record TrackName='See What the Lord Has Done'>]


![](../docs/images/Q1.png)

##### Q2: Given a list of artists, return tracks sorted by the occurrence at album level

In [6]:
q2_query = """
MATCH (artist:Artist)<-[:COMPOSED_BY]-(track:Track)-[:PART_OF]->(album:Album)
WHERE artist.name IN ['Jenifer', 'Jessie J', 'Frank Morgan']
WITH album, COUNT(DISTINCT artist) AS artistsCount
ORDER BY artistsCount DESC
MATCH (album)-[:PART_OF]-(track:Track)
RETURN track.name AS TrackName, album.name AS Album ,artistsCount
"""
q2_results = execute_query(q2_query)
print("\nQ2 Results:", q2_results)


Q2 Results: [<Record TrackName='Sweet Talker' Album='Sugar Sweet' artistsCount=3>, <Record TrackName='Sur Le Fil' Album='Sur Le Fil' artistsCount=1>, <Record TrackName="Walkin'" Album='Reflections' artistsCount=1>]


![](../docs/images/Q2.png)

##### Q3: Given a set of tracks, return artists based on similarity at metadata level

In [7]:
q3_query = """
MATCH (track:Track)-[:HAS_GENRE]->(genre:Genre)<-[:HAS_GENRE]-(similarTrack:Track),
      (similarTrack)-[:COMPOSED_BY]->(artist:Artist)
WHERE track.name IN ['See What the Lord Has Done', 'Sing Your Blues Away']
RETURN DISTINCT artist.name AS ArtistName
"""
q3_results = execute_query(q3_query)
print("Q3 Results:", q3_results)

Q3 Results: [<Record ArtistName='The Staple Singers'>, <Record ArtistName='Todd Warren'>, <Record ArtistName='Enstrümantal'>, <Record ArtistName='JuJu Song'>, <Record ArtistName='Nathaniel Bassey'>, <Record ArtistName='Hillsong en Español, Hillsong Young & Free'>, <Record ArtistName='Sterling Roberts'>, <Record ArtistName='Mark Bishop'>, <Record ArtistName='Dr Tumi'>, <Record ArtistName='Stan Whitmire'>, <Record ArtistName='BarlowGirl'>, <Record ArtistName='Barbara Jones'>]


![](../docs/images/Q3.png)

##### Q4: Given a track return other tracks (from the same artist and/or similar artists that come from spotify_related_artists_ids)

This is a two step query. 
1. First we need to find the artist having more than one track 
2. For similar, we need to create another query


Q4.1: Find other tracks by the same artist

In [8]:
q4_1_results = """
MATCH (artist:Artist)<-[:COMPOSED_BY]-(track:Track)
WITH artist, COLLECT(track) AS tracks
WHERE SIZE(tracks) > 1
UNWIND tracks AS otherTrack
RETURN artist.name AS ArtistName, otherTrack.name AS TrackName

"""
q4_1_results = execute_query(q4_1_results)
print("Q Results:", q4_1_results)

Q Results: [<Record ArtistName='Cerrone, The Reflex' TrackName='Look for Love - The Reflex Revision - Edit'>, <Record ArtistName='Cerrone, The Reflex' TrackName='Diskette - The Reflex Re-Vision Long Version'>, <Record ArtistName='Eric Milnes, Monika Mauch, Harry Van Der Kamp, Franziska Gottwald, Charles Daniels, Montréal Baroque, Johann Sebastian Bach' TrackName='Mein Gott, wie lang, ach lange, BWV 155: Aria: Du musst glauben, du musst hoffen (Alto, Tenor)'>, <Record ArtistName='Eric Milnes, Monika Mauch, Harry Van Der Kamp, Franziska Gottwald, Charles Daniels, Montréal Baroque, Johann Sebastian Bach' TrackName='Suite française No. 4 in E-Flat Major, BWV 815: IV. Gavotte'>, <Record ArtistName='Eric Milnes, Monika Mauch, Harry Van Der Kamp, Franziska Gottwald, Charles Daniels, Montréal Baroque, Johann Sebastian Bach' TrackName='Lute Suite in E Minor, BWV 996 (Arr. J. Williams for Guitar): VI. Gigue'>, <Record ArtistName='Eric Milnes, Monika Mauch, Harry Van Der Kamp, Franziska Gottwald,

![](../docs/images/Q4_1.png)

Q4.2: Find tracks by artists similar based on shared genres

In [9]:
q4_2_results = """
MATCH (originalTrack:Track {name: 'Highway Junkie'})-[:HAS_GENRE]->(genre:Genre)<-[:HAS_GENRE]-(similarTrack:Track),
      (similarTrack)-[:COMPOSED_BY]->(similarArtist:Artist)
WHERE originalTrack.name <> similarTrack.name
RETURN similarArtist.name AS SimilarArtistName, COLLECT(similarTrack.name) AS SimilarTrackNames
"""
q4_2_results = execute_query(q4_2_results)
print("Q Results:", q4_2_results)

Q Results: [<Record SimilarArtistName='John Hartford' SimilarTrackNames=['Boogie']>, <Record SimilarArtistName='Greg Brown' SimilarTrackNames=['Canned Goods']>, <Record SimilarArtistName='Kim Richey' SimilarTrackNames=["I'm Alright"]>, <Record SimilarArtistName='Bobby Helms' SimilarTrackNames=["I Don't Owe You Nothing"]>, <Record SimilarArtistName='Randy Travis' SimilarTrackNames=['King Of The Road']>, <Record SimilarArtistName='Dan Hicks & His Hot Licks' SimilarTrackNames=["I Don't Want Love"]>, <Record SimilarArtistName='The National' SimilarTrackNames=['Conversation 16']>, <Record SimilarArtistName='The Long Ryders' SimilarTrackNames=['I Had a Dream']>, <Record SimilarArtistName='Jake Blount' SimilarTrackNames=['Done Gone']>, <Record SimilarArtistName='Slim Dusty' SimilarTrackNames=['Natural High']>, <Record SimilarArtistName='Mitchell Kersley' SimilarTrackNames=["I Only Cry When I'm Drinking"]>, <Record SimilarArtistName='Willie Nelson/Merle Haggard' SimilarTrackNames=['Pancho And 

![](../docs/images/Q4_2.png)

## Q4_A Given a Track, find related Artists - 1st Level Traversal

In [12]:
q4_A_results = """
MATCH (track:Track {name: "Night Time Is the Right Time"})-[:COMPOSED_BY]->(artist:Artist)
WITH artist, track
MATCH (artist)-[:RELATED_TO]->(related:Artist)<-[:COMPOSED_BY]-(relatedTrack:Track)
RETURN track.name, artist.spotifyId AS ArtistSpotifyID, collect(related.spotifyId) AS SimilarArtistSpotifyIDs, collect(relatedTrack.name) AS SimilarTracks
"""
q4_A_results = execute_query(q4_A_results)
print("Q Results:", q4_A_results)

Q Results: [<Record track.name='Night Time Is the Right Time' ArtistSpotifyID='1eYhYunlNJlDoQhtYBvPsi' SimilarArtistSpotifyIDs=['7guDJrEfX3qb6FEbdPA5qi', '5V0MlUE1Bft0mbLlND7FJz', '4y6J8jwRAwO4dssiSmN91R'] SimilarTracks=['What A Wonderful World', "I Can't Get Started", "(I'm Your) Hoochie Coochie Man"]>]


## Q4_B Given a Track, find related Artists - Second Level Traversal

In [13]:
q4_B_results = """
MATCH (track:Track {name: "Night Time Is the Right Time"})-[:COMPOSED_BY]->(artist:Artist)
WITH artist, track
MATCH (artist)-[:RELATED_TO*2..2]->(related:Artist)<-[:COMPOSED_BY]-(relatedTrack:Track)
RETURN track.name, artist.spotifyId AS ArtistSpotifyID, collect(related.spotifyId) AS SimilarArtistSpotifyIDs, collect(relatedTrack.name) AS SimilarTracks
"""
q4_B_results = execute_query(q4_B_results)
print("Q Results:", q4_B_results)

Q Results: [<Record track.name='Night Time Is the Right Time' ArtistSpotifyID='1eYhYunlNJlDoQhtYBvPsi' SimilarArtistSpotifyIDs=['0Wxy5Qka8BN9crcFkiAxSR', '3koiLjNrgRTNbOwViDipeA'] SimilarTracks=['Three Hundred Pounds Of Joy', 'Change What You Can']>]


In [14]:
#Q5: Given a set of tracks, return radios
q5_query = """
MATCH (track:Track)-[:PLAYS]-(radio:Radio)
WHERE track.name IN ['No Goodbyes', 'Around the World in 80 Days']
RETURN DISTINCT radio.name AS RadioName

"""
q5_results = execute_query(q5_query)
print("Q Results:", q5_results)

Q Results: [<Record RadioName='Outlaw Country'>, <Record RadioName='WHOM 94.9 HOM'>]


![](../docs/images/Q5.png)

In [ ]:
# Close the connection
driver.close()

#### Additional Queries


##### To find other artists who have tracks in the same genres as a given set of tracks:

```sql
MATCH (track:Track)-[:HAS_GENRE]->(genre:Genre)<-[:HAS_GENRE]-(otherTrack:Track)
WHERE track.name IN ['Natural High', 'I Had a Dream'] AND track.name <> otherTrack.name
WITH DISTINCT genre, otherTrack
MATCH (otherTrack)-[:COMPOSED_BY]->(otherArtist:Artist)
RETURN genre.name AS GenreName, otherArtist.name AS OtherArtistName, COLLECT(otherTrack.name) AS OtherTracks
```

![](../docs/images/Q6.png)


##### To recommend tracks for a radio based on the genre:
```sql
MATCH (radio:Radio)-[:PLAYS]->(track:Track)-[:HAS_GENRE]->(genre:Genre)
WHERE radio.name = 'bigFM Balkan'
WITH genre
MATCH (genre)<-[:HAS_GENRE]-(recommendedTrack:Track)
RETURN genre.name AS GenreName, COLLECT(recommendedTrack.name) AS RecommendedTracks
```
![](../docs/images/Q7.png)

_Based on Album, Genre, there can be more....._